In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,MinMaxScaler

In [5]:
df_daily_calendar= pd.read_csv("daily_calendar_with_events.csv")
df_item_prices=pd.read_csv("item_prices.csv")
df_item_sales=pd.read_csv("item_sales.csv")



# Analisis de Prices

In [64]:
df_item_prices.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858


In [57]:
df_item_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [56]:
df_item_prices.isnull()

,item,category,store_code,yearweek,sell_price
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
6965701,False,False,False,True,False
6965702,False,False,False,True,False
6965703,False,False,False,True,False
6965704,False,False,False,True,False


In [93]:
df_item_prices.isnull().sum()

item               0
category           0
store_code         0
yearweek      243920
sell_price         0
dtype: int64

# Analisis Calendar

In [7]:
df_daily_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [8]:
df_daily_calendar.isnull().any()

date           False
weekday        False
weekday_int    False
d              False
event           True
dtype: bool

Vemos que tenemos varios nulos en eventos ya que de 1913 elementos non null en las otras 3 columnas, event tiene solo 26 non null.
Ademas, es la única que da TRUE. 

In [9]:
df_daily_calendar.isnull().sum()

date              0
weekday           0
weekday_int       0
d                 0
event          1887
dtype: int64

In [10]:
df_daily_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [11]:
(df_daily_calendar.isnull().sum() / len(df_daily_calendar)) * 100

date            0.000000
weekday         0.000000
weekday_int     0.000000
d               0.000000
event          98.640878
dtype: float64

Tiene un 98 % de nulos, cuanto influye? Deberiamos hacer un ONE-HOT-ENCONDING ya que eventos seguramente influya en las ventas y en la asignacion de stocks. 

In [12]:
df_daily_calendar[["event"]].value_counts()

event         
SuperBowl         6
Easter            5
NewYear           5
Ramadan starts    5
Thanksgiving      5
Name: count, dtype: int64

In [13]:
df_daily_calendar[["weekday", "weekday_int","date"]].head(7)

,weekday,weekday_int,date
0,Saturday,1,2011-01-29
1,Sunday,2,2011-01-30
2,Monday,3,2011-01-31
3,Tuesday,4,2011-02-01
4,Wednesday,5,2011-02-02
5,Thursday,6,2011-02-03
6,Friday,7,2011-02-04


En cuanto a los dias, deberiamos de separar las fechas para poder trabajarlas por separada. Ademas, eliminar el weekday ya que esta asignado con weekday_int (sabado=1 - Viernes=7)

In [14]:
df_daily_calendar.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1908    False
1909    False
1910    False
1911    False
1912    False
Length: 1913, dtype: bool

In [15]:
df_daily_calendar.duplicated().sum().any()

np.False_

No tiene duplicados

# Analisis de Sales

In [16]:
df_item_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


Tiene 1920 columnas y 1913 entradas (representadas por cada accesorio y cada store code/region)

In [17]:
df_item_sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [18]:
df_item_sales.isnull().sum()

id            0
item          0
category      0
department    0
store         0
             ..
d_1909        0
d_1910        0
d_1911        0
d_1912        0
d_1913        0
Length: 1920, dtype: int64

In [19]:
df_item_sales.isnull().sum().any()

np.False_

No tiene nulos

In [20]:
df_item_sales.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
30485    False
30486    False
30487    False
30488    False
30489    False
Length: 30490, dtype: bool

In [21]:
df_item_sales.duplicated().sum()

np.int64(0)

No tiene duplicados

# Trabajo de tablas 

Luego del analizar las tablas con los companeros, hemos llegado a la conclucion que, para poder trabajarlas y poder unirlas, deberiamos transformas cada dia (d_1,d_2 ..) de la tabla "Sales" a una fila. Esto, nos va ayudar a unir a la tablas. Podremos unir las dates, weekday_int, si tiene evento o no, etc.

### Trabajo tabla Calendar

Primero hacemos una copia para asegurarnos que si la cagamos estamos cubiertos

In [22]:
calendar= df_daily_calendar.copy()

Ahora no, pero mas adelante deberiamos eliminar los weekday ya que cada dia esta asignado por un weeday_int.

In [23]:
calendar

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN
...,...,...,...,...,...
1908,2016-04-20,Wednesday,5,d_1909,NaN
1909,2016-04-21,Thursday,6,d_1910,NaN
1910,2016-04-22,Friday,7,d_1911,NaN
1911,2016-04-23,Saturday,1,d_1912,NaN


Separamos las fechas para crear nuevas variables y analizar las tendencias mas facilmente. Podemos analizarla por mes y año. Para esto, primero debemos transformar la columna en tipo datetime.

In [24]:
calendar['date'] = pd.to_datetime(calendar['date'])

In [25]:
calendar['year'] = calendar['date'].dt.year
calendar['month'] = calendar['date'].dt.month
calendar['day'] = calendar['date'].dt.day
print(calendar)

           date    weekday  weekday_int       d event  year  month  day
0    2011-01-29   Saturday            1     d_1   NaN  2011      1   29
1    2011-01-30     Sunday            2     d_2   NaN  2011      1   30
2    2011-01-31     Monday            3     d_3   NaN  2011      1   31
3    2011-02-01    Tuesday            4     d_4   NaN  2011      2    1
4    2011-02-02  Wednesday            5     d_5   NaN  2011      2    2
...         ...        ...          ...     ...   ...   ...    ...  ...
1908 2016-04-20  Wednesday            5  d_1909   NaN  2016      4   20
1909 2016-04-21   Thursday            6  d_1910   NaN  2016      4   21
1910 2016-04-22     Friday            7  d_1911   NaN  2016      4   22
1911 2016-04-23   Saturday            1  d_1912   NaN  2016      4   23
1912 2016-04-24     Sunday            2  d_1913   NaN  2016      4   24

[1913 rows x 8 columns]


Generamos la columna de week

In [ ]:
calendar_1['date'] = pd.to_datetime(calendar_1[['year', 'month', 'day']])  
calendar_1['week'] = calendar_1['date'].dt.isocalendar().week  
calendar_1

,weekday,weekday_int,d,event,year,month,day,date,week
0,Saturday,1,d_1,NaN,2011,1,29,2011-01-29,4
1,Sunday,2,d_2,NaN,2011,1,30,2011-01-30,4
2,Monday,3,d_3,NaN,2011,1,31,2011-01-31,5
3,Tuesday,4,d_4,NaN,2011,2,1,2011-02-01,5
4,Wednesday,5,d_5,NaN,2011,2,2,2011-02-02,5
...,...,...,...,...,...,...,...,...,...
1908,Wednesday,5,d_1909,NaN,2016,4,20,2016-04-20,16
1909,Thursday,6,d_1910,NaN,2016,4,21,2016-04-21,16
1910,Friday,7,d_1911,NaN,2016,4,22,2016-04-22,16
1911,Saturday,1,d_1912,NaN,2016,4,23,2016-04-23,16


copiamos por si la cagamos

In [48]:
calendar_1=calendar.copy()
calendar_1

,weekday,weekday_int,d,event,year,month,day
0,Saturday,1,d_1,NaN,2011,1,29
1,Sunday,2,d_2,NaN,2011,1,30
2,Monday,3,d_3,NaN,2011,1,31
3,Tuesday,4,d_4,NaN,2011,2,1
4,Wednesday,5,d_5,NaN,2011,2,2
...,...,...,...,...,...,...,...
1908,Wednesday,5,d_1909,NaN,2016,4,20
1909,Thursday,6,d_1910,NaN,2016,4,21
1910,Friday,7,d_1911,NaN,2016,4,22
1911,Saturday,1,d_1912,NaN,2016,4,23


Eliminamos la columna date.

In [51]:
calendar_1.drop(columns="date", inplace=True)

In [37]:
calendar.drop(columns=["date"],inplace=True)

### Trabajo Tabla Sales

Como mencionamos anteriormente, debemos pasar cada dia como un elemento para poder unirla a las otras tablas. 

In [27]:
df_item_sales

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,2,...,2,0,0,0,0,0,1,0,0,1
30486,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,6,0,...,2,1,0,2,0,1,0,0,1,0
30488,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,0,...,0,0,1,0,0,1,0,3,1,3


Hacemos una copia por si la cagamos

In [28]:
sales = df_item_sales.copy()

Vamos a usar la funcion "melt" que convierte una tabla de formato wide (ancho) a long (largo), donde cada día (d_#) se convierte en una nueva fila con su valor correspondiente.

In [29]:
sales =sales.melt(
    id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'],  # Columnas que no cambian
    var_name='day',   # Nuevo nombre de columna para los días
    value_name='sales' # Nuevo nombre de columna para los valores de ventas
)

print(sales)

                               id               item     category  \
0          ACCESORIES_1_001_NYC_1   ACCESORIES_1_001   ACCESORIES   
1          ACCESORIES_1_002_NYC_1   ACCESORIES_1_002   ACCESORIES   
2          ACCESORIES_1_003_NYC_1   ACCESORIES_1_003   ACCESORIES   
3          ACCESORIES_1_004_NYC_1   ACCESORIES_1_004   ACCESORIES   
4          ACCESORIES_1_005_NYC_1   ACCESORIES_1_005   ACCESORIES   
...                           ...                ...          ...   
58327365  SUPERMARKET_3_823_PHI_3  SUPERMARKET_3_823  SUPERMARKET   
58327366  SUPERMARKET_3_824_PHI_3  SUPERMARKET_3_824  SUPERMARKET   
58327367  SUPERMARKET_3_825_PHI_3  SUPERMARKET_3_825  SUPERMARKET   
58327368  SUPERMARKET_3_826_PHI_3  SUPERMARKET_3_826  SUPERMARKET   
58327369  SUPERMARKET_3_827_PHI_3  SUPERMARKET_3_827  SUPERMARKET   

             department              store store_code        region     day  \
0          ACCESORIES_1  Greenwich_Village      NYC_1      New York     d_1   
1          AC

In [30]:
sales.head()

,id,item,category,department,store,store_code,region,day,sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0


In [31]:
sales = sales.rename(columns={'day': 'd'})

In [42]:
sales.head()

,id,item,category,department,store,store_code,region,d,sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0


In [41]:
calendar.head()

,weekday,weekday_int,d,event,year,month,day
0,Saturday,1,d_1,NaN,2011,1,29
1,Sunday,2,d_2,NaN,2011,1,30
2,Monday,3,d_3,NaN,2011,1,31
3,Tuesday,4,d_4,NaN,2011,2,1
4,Wednesday,5,d_5,NaN,2011,2,2


# Trabajo Prices

Trabajamos la columna yearweek para poder separar el año y la semana. 

In [91]:
df_item_prices["yearweek"] = df_item_prices["yearweek"]/100
df_item_prices

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.28,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.29,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.30,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.31,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.32,10.9858
...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000


# Unir DF Sales y Calendar

Para comenzar con el analisis, debemos unir los DF. Para unirlos, vamos a utilizar la funcion "merge" y los vamos a unificar a traves de un INNER JOIN on "d".

In [72]:
merge_sales_calendar = pd.merge(sales,calendar_1,on='d', how='inner')

In [90]:
merge_sales_calendar

,id,item,category,department,store,store_code,region,d,sales,weekday,weekday_int,event,year,month,day,week
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,Saturday,1,NaN,2011,1,29,4
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,Saturday,1,NaN,2011,1,29,4
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,Saturday,1,NaN,2011,1,29,4
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,Saturday,1,NaN,2011,1,29,4
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,Saturday,1,NaN,2011,1,29,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,Sunday,2,NaN,2016,4,24,16
58327366,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,Sunday,2,NaN,2016,4,24,16
58327367,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,Sunday,2,NaN,2016,4,24,16
58327368,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,3,Sunday,2,NaN,2016,4,24,16


In [78]:
df_item_prices["yearweek"].max()

np.float64(201617.0)

In [92]:
df_item_prices

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.28,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.29,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.30,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.31,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.32,10.9858
...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000


In [112]:
df_item_prices

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.28,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.29,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.30,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.31,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.32,10.9858
...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000


In [113]:
prices_test_2=df_item_prices.fillna(0000.00)
prices_test_2

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.28,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.29,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.30,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.31,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.32,10.9858
...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,0.00,1.2000
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,0.00,1.2000
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,0.00,1.2000
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,0.00,1.2000


In [110]:
prices_test["yearweek"] = prices_test["yearweek"].fillna('').astype(str)
prices_test["year"] = prices_test["yearweek"].str[:4]
prices_test["week"] = prices_test["yearweek"].str[5:]

In [111]:
prices_test

,item,category,store_code,yearweek,sell_price,year,week
0,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.28,12.7414,2013,28
1,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.29,12.7414,2013,29
2,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.3,10.9858,2013,3
3,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.31,10.9858,2013,31
4,ACCESORIES_1_001,ACCESORIES,NYC_1,2013.32,10.9858,2013,32
...,...,...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,0.0,1.2000,0.0,
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,0.0,1.2000,0.0,
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,0.0,1.2000,0.0,
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,0.0,1.2000,0.0,


In [114]:
prices_test["week"].isnull()

0          False
1          False
2          False
3          False
4          False
           ...  
6965701    False
6965702    False
6965703    False
6965704    False
6965705    False
Name: week, Length: 6965706, dtype: bool

In [70]:
merge_sales_calendar.isnull().any()

id             False
item           False
category       False
department     False
store          False
store_code     False
region         False
d              False
sales          False
weekday        False
weekday_int    False
event           True
year           False
month          False
day            False
dtype: bool

In [ ]:
merge_sales_calendar

,id,item,category,department,store,store_code,region,d,sales,weekday,weekday_int,event,year,month,day,week
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,Saturday,1,NaN,2011,1,29,4
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,Saturday,1,NaN,2011,1,29,4
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,Saturday,1,NaN,2011,1,29,4
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,Saturday,1,NaN,2011,1,29,4
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,Saturday,1,NaN,2011,1,29,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,Sunday,2,NaN,2016,4,24,16
58327366,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,Sunday,2,NaN,2016,4,24,16
58327367,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,Sunday,2,NaN,2016,4,24,16
58327368,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,3,Sunday,2,NaN,2016,4,24,16


In [ ]:
sales_calendar_prices = pd.merge(merge_sales_calendar,df_item_prices,on='d', how='inner')